# Code for scraping data from VoteWatch

In [ ]:
from urllib.request import Request, urlopen

import pandas as pd

In [ ]:
def scrapeWebsite(url):
    #Website to be requested
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    xhtml = urlopen(req).read()

    string = xhtml.decode('utf-8')
    title = string.split('<div class="div_tabled_info_full">')[1].split('<')[0]
    new_url = string.split('var source_individual_votes = "')[1].split('"')[0]

    req = Request(new_url, headers={'User-Agent': 'Mozilla/5.0'})
    xhtml = urlopen(req).read()

    df = pd.read_json(xhtml, orient='records')

    df['MEP'] = df['all_votes'].astype(str).str.split("mep_name_link': 'https://www.votewatch.eu/en/term9-").str[1].str.split(".html").str[0].str.replace("-", " ").str.title()
    df['group_abbv'] = df['all_votes'].astype(str).str.split("grup': '").str[1].str.split("'").str[0]
    df['country'] = df['all_votes'].astype(str).str.split("euro_tara_nume': '").str[1].str.split("'").str[0]
    df[title] = df['all_votes'].astype(str).str.split("euro_vot_valoare_text': '").str[1].str.split("'").str[0]
    df = df.drop(['all_votes'], axis=1)
    return df

In [ ]:
# website for motion to be scraped- this was adapted for the motions used
website = 'https://www.votewatch.eu/en/term9-the-impacts-of-climate-change-on-vulnerable-populations-in-developing-countries-motion-for-resolutio.html'

In [ ]:
# range updated based on number of votes
for i in range(1,45):
    if i == 1:
        url = website
    else:
        url = website[:-5] + '-' + str(i) + website[-5:]
    
    newdf = scrapeWebsite(url)
    
    if i == 1:
        df = newdf
    else:
        df = df.merge(newdf, on=['MEP', 'group_abbv', 'country'])

# Fixing data issue
df = df.replace('Didn&#39;t vote', "Didn't vote") 
df.head()

,MEP,group_abbv,country,"Paragraph 1, amendment 5","Paragraph 3, amendment 6",Paragraph 6/1,Paragraph 6/2,"Paragraph 16, indent 5, amendment 7",Paragraph 18,Paragraph 19,Paragraph 20,"Paragraph 24, amendment 8","Paragraph 27, amendment 9","Paragraph 29, amendment 10",Paragraph 52/1,Paragraph 52/2,Paragraph 69/1,Paragraph 69/2,"Paragraph 73, intro, amendment 11","Paragraph 73, indent 1, amendment 12","Paragraph 73, indent 3, amendment 13","Paragraph 73, indent 3, amendment 4","Paragraph 73, indent 4/1","Paragraph 73, indent 4/2","Paragraph 73, indent 5, amendment 15","Paragraph 73, after indent 5, amendment 24","Paragraph 74, intro, amendment 16","Paragraph 74, indent 1, amendment 17","Paragraph 74, indent 3, amendment 18","Paragraph 74, after indent 3, amendment 25","Paragraph 75, amendment 19",Paragraph 75/1,Paragraph 75/2,"Paragraph 76, amendment 20","Paragraph 77, amendment 21","Paragraph 78, amendment 22","Paragraph 80, amendment 23",Citation 3/1,Citation 3/2,"After citation 4, amendment 1","Recital G, amendment 2",Recital J/1,Recital J/2,Recital J/3,"Recital V, amendment 3","Recital AE, amendment 4",vote: resolution (as a whole)
0,Rasmus Andresen,Greens/EFA,Germany,Against,Against,For,For,Against,For,For,For,Against,For,Against,For,For,For,For,For,Against,Against,Against,For,For,For,For,Against,Against,Against,For,Against,For,For,Against,For,Against,Against,For,For,For,Against,For,For,For,Against,Against,Abstain
1,Michael Bloss,Greens/EFA,Germany,Against,Against,For,For,Against,For,For,For,Against,For,Against,For,For,For,For,For,Against,Against,Against,For,For,For,For,Against,Against,Against,For,Against,For,For,Against,For,Against,Against,For,For,For,Against,For,For,For,Against,Against,Abstain
2,Reinhard Butikofer,Greens/EFA,Germany,Against,Against,For,For,Against,For,For,For,Against,For,Against,For,For,For,For,For,Against,Against,Against,For,For,For,For,Against,Against,Against,For,Against,For,For,Against,For,Against,Against,For,For,For,Against,For,For,For,Against,Against,Abstain
3,Anna Cavazzini,Greens/EFA,Germany,Against,Against,For,For,Against,For,For,For,Against,For,Against,For,For,For,For,For,Against,Against,Against,For,For,For,For,Against,Against,Against,For,Against,For,For,Against,For,Against,Against,For,For,For,Against,For,For,For,Against,Against,Abstain
4,Anna Deparnay Grunenberg,Greens/EFA,Germany,Against,Against,For,For,Against,For,For,For,Against,For,Against,For,For,For,For,For,Against,Against,Against,For,For,For,For,Against,Against,Against,For,Against,For,For,Against,For,Against,Against,For,For,For,Against,For,For,For,Against,Against,Abstain


In [ ]:
# Saving results to csv
df.to_csv(r'mepdata.csv', index = False)

In [ ]:
# Checking number of rows
df_len = len(df)
if df_len == 705:
    print("The dataset has {} rows which equals the expected 705 MEPs".format(df_len))
else:
    print("The dataset has {} rows which is not equal to the  expected 705 MEPs".format(df_len))

# Checking for missing data
if df.isnull().values.any() | df.eq('').values.any():
  print('The dataset has missing values')
else:
  print('The dataset has no missing values')

# Checking no issues with duplicates
if df["MEP"].is_unique:
   print('The MEP column has no duplicates')
else:
  print('There are duplicate values in the MEP column')

The dataset has 705 rows which equals the expected 705 MEPs
The dataset has no missing values
The MEP column has no duplicates
